# ICO projects from coinmarketcap 

An initial coin offering (ICO) is an event where a company sells a new cryptocurrency to raise money. Investors receive cryptocurrency in exchange for their financial contributions. In many ways, an ICO is the cryptocurrency version of an initial public offering (IPO) in the stock market.

There are milions of ICOs was created, only some project have enough credibility could show them off on coinmarketcap. Let's see how were their perfomances?

Data from coinmarketcap: https://coinmarketcap.com/ico-calendar/ended/

In [119]:
import json


icos = json.load(open('./pj_data/w6.project.cmc.json'))
icoList = icos['data']['icoList']

prices = json.load(open('./pj_data/price_data.json'))


There are 1220 icos in this dataset

In [120]:
len(icoList)

1220

Let's take a look at data structure

In [121]:
icoList[0]

{'cryptoId': 13745,
 'icoPriceUsd': 0.08,
 'currentStage': 'IDO',
 'end': '2021-11-24T02:00:00.000Z',
 'goalUsd': 98000,
 'launchPad': {'exchangeName': 'Redkite',
  'launchpadUrl': 'https://redkite.polkafoundry.com/#/buy-token/109'},
 'crypto': {'id': 13745,
  'name': 'Polygonum Online',
  'symbol': 'POG',
  'slug': 'polygonum-online',
  'logo': 'https://s2.coinmarketcap.com/static/img/coins/64x64/13745.png',
  'tags': ['gaming', 'play-to-earn'],
  'contracts': [{'id': 1839, 'name': 'BNB Smart Chain (BEP20)'}]}}

I will restructure json to be easier to convert to Pandas DataFrame

In [122]:
datalist = []
common_contracts = {'Ethereum':'eth','BNB Smart Chain (BEP20)':'bnb','Solana':'sol','Polygon':'matic','Avalanche C-Chain':'avax','Cardano':'ada'}

for item in icoList:
    price = prices.get(str(item.get('cryptoId')))
    if (price):
        for price_key in ['date_added','circulating_supply','total_supply','is_active','infinite_supply','last_updated']:
            if (price.get(price_key)):
                item[price_key] = price[price_key]
        quote = price.get('quote')
        if (quote):
            usd = quote.get('USD')
            item['price'] = usd.get('price')
            item['last_updated'] = usd.get('last_updated')
        if (not item.get('is_active')):
            item['is_active']=False
        else:
            item['is_active']=True
    launchPad = item.get('launchPad')
    if (launchPad):
        for k in launchPad:
            item[k] = launchPad[k]
        del item['launchPad'] 
    
    crypto = item.get('crypto')
    if (crypto):
        for k in crypto:
            item[k] = crypto[k]
        contracts = crypto.get('contracts')
        if (contracts):
            item['n_contracts'] = len(contracts)
            ids = []
            names = []
            for contract in contracts:
                id = contract['id']
                ids.append(id)
                name = contract['name']
                names.append(name)
                if (name in common_contracts):
                    item['is'+common_contracts.get(name)] = True

            item['contract_ids'] =  ','.join(str(v) for v in ids)
            item['contract_names'] = ','.join(names)
            del item['contracts']
        del item['crypto']
        if (item.get('tags')):
            tags = item.get('tags')
            item['tags'] = ','.join(tags)
            # 
            if (len(tags)>1):
                for bnbstr in ['bnb-chain','binance-chain','binance-smart-chain']:
                    if (bnbstr in tags):
                        tags.remove(bnbstr)
            if ((len(tags)>1) & ('gaming' in tags)):
                tags=['gaming']
            if ((len(tags)>1) & ('collectibles-nfts' in tags)):
                tags=['collectibles-nfts']
            if ((len(tags)>1) & ('polygon-ecosystem' in tags)):
                tags=['polygon-ecosystem']
            if ((len(tags)>1) & ('ai-big-data' in tags)):
                tags=['ai-big-data']
            if ((len(tags)>1) & ('defi' in tags)):
                tags=['defi']
            item['tags_reduce'] = ','.join(tags)
    
    

In [123]:
icoList[0]

{'cryptoId': 13745,
 'icoPriceUsd': 0.08,
 'currentStage': 'IDO',
 'end': '2021-11-24T02:00:00.000Z',
 'goalUsd': 98000,
 'date_added': '2021-11-30T05:20:03.000Z',
 'total_supply': 100000000,
 'is_active': True,
 'last_updated': '2023-09-17T09:11:00.000Z',
 'price': 0.005751353610902459,
 'exchangeName': 'Redkite',
 'launchpadUrl': 'https://redkite.polkafoundry.com/#/buy-token/109',
 'id': 13745,
 'name': 'Polygonum Online',
 'symbol': 'POG',
 'slug': 'polygonum-online',
 'logo': 'https://s2.coinmarketcap.com/static/img/coins/64x64/13745.png',
 'tags': 'gaming,play-to-earn',
 'n_contracts': 1,
 'isbnb': True,
 'contract_ids': '1839',
 'contract_names': 'BNB Smart Chain (BEP20)',
 'tags_reduce': 'gaming'}

In [124]:
import pandas as pd

df = pd.DataFrame(icoList)

df.head()

,cryptoId,icoPriceUsd,currentStage,end,goalUsd,date_added,total_supply,is_active,last_updated,price,...,tags_reduce,start,iseth,issol,ismatic,circulating_supply,isada,isavax,infinite_supply,launchPad
0,13745,0.08,IDO,2021-11-24T02:00:00.000Z,98000.0,2021-11-30T05:20:03.000Z,100000000.0,True,2023-09-17T09:11:00.000Z,0.005751,...,gaming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22621,0.03,IDO,2023-08-31T00:00:00.000Z,80000.0,2022-11-09T23:52:00.000Z,NaN,False,2023-09-17T09:13:17.683Z,0.000000,...,NaN,2023-08-24T00:00:00.000Z,True,True,NaN,NaN,NaN,NaN,NaN,NaN
2,24858,0.60,IDO,2023-08-26T00:00:00.000Z,1500000.0,2023-05-02T00:57:29.000Z,100000000.0,True,2023-09-17T09:11:00.000Z,0.055907,...,NaN,2023-08-23T12:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,23920,NaN,ICO,2023-08-18T12:00:00.000Z,NaN,2023-09-12T13:23:42.000Z,NaN,True,2023-09-17T09:11:00.000Z,0.011018,...,NaN,2023-08-15T13:00:00.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,27522,0.95,IDO,NaN,500000.0,2023-07-08T17:11:22.000Z,NaN,False,2023-09-17T09:13:17.683Z,0.000000,...,NaN,2023-08-10T00:00:00.000Z,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Top tags are gaming, collectibles-nfts, defi, bnb-chain, polygon-ecosystem,vbc-ventures-portfolio, ai-big-data,

In [125]:
tags_list = df['tags'].value_counts()
tags_list.to_csv('./pj_data/tags.csv')

In [126]:
tags_list = df['tags_reduce'].value_counts()
tags_list.to_csv('./pj_data/tags_reduce.csv')

Top contract are BNB, ETH, Solana, Polygon, Avalanche, Cardano

In [127]:
counts = df['contract_names'].value_counts()
counts.to_csv('./pj_data/contract_names.csv')

Remove some unused columns

In [128]:
#remove unused columns
new_df = df.drop(['launchpadUrl', 'tags', 'contract_ids', 'contract_names','slug','logo','cryptoId','id','launchpadUrl','launchPad','infinite_supply','circulating_supply'], axis=1)
new_df.head()

,icoPriceUsd,currentStage,end,goalUsd,date_added,total_supply,is_active,last_updated,price,exchangeName,...,symbol,n_contracts,isbnb,tags_reduce,start,iseth,issol,ismatic,isada,isavax
0,0.08,IDO,2021-11-24T02:00:00.000Z,98000.0,2021-11-30T05:20:03.000Z,100000000.0,True,2023-09-17T09:11:00.000Z,0.005751,Redkite,...,POG,1.0,True,gaming,NaN,NaN,NaN,NaN,NaN,NaN
1,0.03,IDO,2023-08-31T00:00:00.000Z,80000.0,2022-11-09T23:52:00.000Z,NaN,False,2023-09-17T09:13:17.683Z,0.000000,Sphere,...,SAX,2.0,NaN,NaN,2023-08-24T00:00:00.000Z,True,True,NaN,NaN,NaN
2,0.60,IDO,2023-08-26T00:00:00.000Z,1500000.0,2023-05-02T00:57:29.000Z,100000000.0,True,2023-09-17T09:11:00.000Z,0.055907,Fjord,...,ALTD,NaN,NaN,NaN,2023-08-23T12:00:00.000Z,NaN,NaN,NaN,NaN,NaN
3,NaN,ICO,2023-08-18T12:00:00.000Z,NaN,2023-09-12T13:23:42.000Z,NaN,True,2023-09-17T09:11:00.000Z,0.011018,DAO maker,...,VV,NaN,NaN,NaN,2023-08-15T13:00:00.000Z,NaN,NaN,NaN,NaN,NaN
4,0.95,IDO,NaN,500000.0,2023-07-08T17:11:22.000Z,NaN,False,2023-09-17T09:13:17.683Z,0.000000,DAO maker,...,PATEX,1.0,NaN,NaN,2023-08-10T00:00:00.000Z,True,NaN,NaN,NaN,NaN


In [129]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1220 entries, 0 to 1219
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   icoPriceUsd   1136 non-null   float64
 1   currentStage  1218 non-null   object 
 2   end           1154 non-null   object 
 3   goalUsd       1009 non-null   float64
 4   date_added    1212 non-null   object 
 5   total_supply  949 non-null    float64
 6   is_active     1212 non-null   object 
 7   last_updated  1212 non-null   object 
 8   price         1212 non-null   float64
 9   exchangeName  1217 non-null   object 
 10  name          1220 non-null   object 
 11  symbol        1220 non-null   object 
 12  n_contracts   1123 non-null   float64
 13  isbnb         771 non-null    object 
 14  tags_reduce   766 non-null    object 
 15  start         1219 non-null   object 
 16  iseth         450 non-null    object 
 17  issol         85 non-null     object 
 18  ismatic       185 non-null  

In [130]:
new_df.columns

Index(['icoPriceUsd', 'currentStage', 'end', 'goalUsd', 'date_added',
       'total_supply', 'is_active', 'last_updated', 'price', 'exchangeName',
       'name', 'symbol', 'n_contracts', 'isbnb', 'tags_reduce', 'start',
       'iseth', 'issol', 'ismatic', 'isada', 'isavax'],
      dtype='object')

In [131]:
new_df.describe().T

,count,mean,std,min,25%,50%,75%,max
icoPriceUsd,1136.0,6.327332e+00,1.928688e+02,0.0,1.500000e-02,4.700000e-02,1.700000e-01,6.500000e+03
goalUsd,1009.0,1.481872e+06,1.096339e+07,0.0,1.000000e+05,1.500000e+05,4.000000e+05,3.000000e+08
total_supply,949.0,1.552132e+14,3.533313e+15,100000.0,1.000000e+08,5.000000e+08,1.000000e+09,1.000000e+17
price,1212.0,7.235836e+00,1.967817e+02,0.0,1.345148e-04,1.884117e-03,1.626796e-02,6.615486e+03
n_contracts,1123.0,1.569902e+00,1.040731e+00,1.0,1.000000e+00,1.000000e+00,2.000000e+00,1.000000e+01


Fill row has null values

In [132]:
# fillna : False with is_platform
new_df['date_added'].fillna(new_df['start'], inplace=True)
new_df['end'].fillna(new_df['start'], inplace=True)
new_df.fillna({'iseth':False, 'issol':False, 'ismatic':False, 'isbnb':False,'isada':False, 'isavax':False, 'exchangeName':'other', 'n_contracts':1,'tags_reduce':'other', 'is_active': False, 'price':0, 'total_supply': 1e+17, 'start': '2021-11-24T02:00:00.000Z', 'last_updated':'2023-09-17T09:11:00.000Z', 'goalUsd':3.000000e+08,'currentStage':'ICO','icoPriceUsd':0}, inplace=True)

Add feature: lifetime = last_updated - date_added

In [133]:


new_df['last_updated'] = pd.to_datetime(new_df['last_updated'])
new_df['date_added'] = pd.to_datetime(new_df['date_added'])

new_df['lifetime'] = new_df['last_updated'] - new_df['date_added']
new_df['timecreated'] = new_df['last_updated'].max() - new_df['date_added']


Drop some more unused fields

In [134]:
new_df1 = new_df.drop(['last_updated','date_added', 'start', 'end','name','symbol'], axis=1)
new_df1.head()

,icoPriceUsd,currentStage,goalUsd,total_supply,is_active,price,exchangeName,n_contracts,isbnb,tags_reduce,iseth,issol,ismatic,isada,isavax,lifetime,timecreated
0,0.08,IDO,98000.0,1.000000e+08,True,0.005751,Redkite,1.0,True,gaming,False,False,False,False,False,656 days 03:50:57,656 days 03:53:25.281000
1,0.03,IDO,80000.0,1.000000e+17,False,0.000000,Sphere,2.0,False,other,True,True,False,False,False,311 days 09:21:17.683000,311 days 09:21:28.281000
2,0.60,IDO,1500000.0,1.000000e+08,True,0.055907,Fjord,1.0,False,other,False,False,False,False,False,138 days 08:13:31,138 days 08:15:59.281000
3,0.00,ICO,300000000.0,1.000000e+17,True,0.011018,DAO maker,1.0,False,other,False,False,False,False,False,4 days 19:47:18,4 days 19:49:46.281000
4,0.95,IDO,500000.0,1.000000e+17,False,0.000000,DAO maker,1.0,False,other,True,False,False,False,False,70 days 16:01:55.683000,70 days 16:02:06.281000


Reduce uncommon tags (count < 5) to others

In [135]:
tagscount = new_df1.tags_reduce.value_counts()
tagscountcheck = new_df1.tags_reduce.isin(tagscount.index[tagscount<5])
new_df1.loc[tagscountcheck, 'tags_reduce'] = 'other'

new_df1.tags_reduce.value_counts()

tags_reduce
other                                 694
gaming                                233
collectibles-nfts                      74
defi                                   67
bnb-chain                              54
polygon-ecosystem                      23
vbc-ventures-portfolio                 19
ai-big-data                            16
okex-blockdream-ventures-portfolio     11
genpad                                  9
bullperks-launchpad                     7
metaverse                               7
dao-maker                               6
Name: count, dtype: int64

Reduce uncommon exchangeName (count <15) to 'Other'

In [136]:
counts = new_df1.exchangeName.value_counts()
counts.to_csv('./pj_data/exchangNameCounts.csv')

ecount = new_df1.exchangeName.value_counts()
ecountcheck = new_df1.exchangeName.isin(ecount.index[ecount<15])
new_df1.loc[ecountcheck, 'exchangeName'] = 'Other'

new_df1.exchangeName.value_counts()

exchangeName
Other                      803
Gate.io                     71
Polkastarter                56
Poolz                       43
Binance                     42
DAO maker                   33
Ignition (Paid Network)     31
Trustpad                    28
Coinlist                    23
Enjinstarter                20
DuckStarter                 20
BSCpad                      19
GameFi                      16
Kommunitas                  15
Name: count, dtype: int64

In [137]:
new_df1.currentStage.value_counts()

counts = new_df1.currentStage.value_counts()
counts.to_csv('./pj_data/currentStageCounts.csv')

scount = new_df1.currentStage.value_counts()
scountcheck = new_df1.currentStage.isin(scount.index[scount<10])
new_df1.loc[scountcheck, 'currentStage'] = 'Other'

new_df1.currentStage.value_counts()

currentStage
IDO             801
IEO             168
ICO             137
Other            52
Private Sale     36
Seed Sale        14
IFO              12
Name: count, dtype: int64

In [138]:
# Get a Pd.Series consisting of all the string categoricals
categories = new_df1.dtypes[new_df1.dtypes == object]  # filtering by string 
categories = categories.index.tolist()
categories


['currentStage', 'exchangeName', 'tags_reduce']

In [139]:

from sklearn.preprocessing import LabelEncoder

le_c = LabelEncoder()
new_df1['currentStage'] = le_c.fit_transform(new_df1.currentStage)
le_e = LabelEncoder()
new_df1['exchangeName'] = le_e.fit_transform(new_df1.exchangeName)
le_t = LabelEncoder()
new_df1['tags_reduce'] = le_t.fit_transform(new_df1.tags_reduce)

new_df1.head()

,icoPriceUsd,currentStage,goalUsd,total_supply,is_active,price,exchangeName,n_contracts,isbnb,tags_reduce,iseth,issol,ismatic,isada,isavax,lifetime,timecreated
0,0.08,1,98000.0,1.000000e+08,True,0.005751,10,1.0,True,6,False,False,False,False,False,656 days 03:50:57,656 days 03:53:25.281000
1,0.03,1,80000.0,1.000000e+17,False,0.000000,10,2.0,False,10,True,True,False,False,False,311 days 09:21:17.683000,311 days 09:21:28.281000
2,0.60,1,1500000.0,1.000000e+08,True,0.055907,10,1.0,False,10,False,False,False,False,False,138 days 08:13:31,138 days 08:15:59.281000
3,0.00,0,300000000.0,1.000000e+17,True,0.011018,3,1.0,False,10,False,False,False,False,False,4 days 19:47:18,4 days 19:49:46.281000
4,0.95,1,500000.0,1.000000e+17,False,0.000000,3,1.0,False,10,True,False,False,False,False,70 days 16:01:55.683000,70 days 16:02:06.281000


In [147]:
le = le_t
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print(le_name_mapping)

{'ai-big-data': 0, 'bnb-chain': 1, 'bullperks-launchpad': 2, 'collectibles-nfts': 3, 'dao-maker': 4, 'defi': 5, 'gaming': 6, 'genpad': 7, 'metaverse': 8, 'okex-blockdream-ventures-portfolio': 9, 'other': 10, 'polygon-ecosystem': 11, 'vbc-ventures-portfolio': 12}


In [141]:
new_df1.describe().T

,count,mean,std,min,25%,50%,75%,max
icoPriceUsd,1220.0,5.89168,186.111892,0.0,0.01,0.04,0.15,6500.0
currentStage,1220.0,1.348361,1.145289,0.0,1.0,1.0,1.0,6.0
goalUsd,1220.0,53110827.091408,113386407.666652,0.0,100000.0,200000.0,2055000.0,300000000.0
total_supply,1220.0,22333850280767668.0,41637138310842208.0,100000.0,120000000.0,1000000000.0,64285009000.0,100000000000000000.0
price,1220.0,7.188388,196.135807,0.0,0.000127,0.00184,0.015824,6615.485707
exchangeName,1220.0,8.909016,2.816933,0.0,10.0,10.0,10.0,13.0
n_contracts,1220.0,1.52459,1.010309,1.0,1.0,1.0,2.0,10.0
tags_reduce,1220.0,7.939344,2.963479,0.0,6.0,10.0,10.0,12.0
lifetime,1220,636 days 22:34:20.097901640,251 days 21:05:19.040350624,4 days 19:47:18,513 days 09:24:17,660 days 13:31:15.500000,819 days 09:11:00,1692 days 13:59:05
timecreated,1220,693 days 17:52:56.384278688,216 days 17:02:20.229671072,4 days 19:49:46.281000,584 days 19:35:34.031000,685 days 05:40:59.781000,842 days 09:13:28.281000,1692 days 14:01:33.281000


We could see average lifetime is 636 days (2 years)

Encode Boolean to 1,0

In [142]:
new_df1 = new_df1*1

In [143]:
new_df1.dtypes

icoPriceUsd             float64
currentStage              int64
goalUsd                 float64
total_supply            float64
is_active                 int64
price                   float64
exchangeName              int64
n_contracts             float64
isbnb                     int64
tags_reduce               int64
iseth                     int64
issol                     int64
ismatic                   int64
isada                     int64
isavax                    int64
lifetime        timedelta64[ns]
timecreated     timedelta64[ns]
dtype: object

Let write down to csv file

In [144]:
new_df1['lifeday']= new_df1['lifetime'].dt.days
new_df1=new_df1.drop(['lifetime'],axis=1)
new_df1['dayscreated']= new_df1['timecreated'].dt.days
new_df1=new_df1.drop(['timecreated'],axis=1)

new_df1=new_df1.reindex(columns=["icoPriceUsd","currentStage","goalUsd","total_supply","price","exchangeName","n_contracts","isbnb","tags_reduce","iseth","issol","ismatic","isada","isavax","lifeday","dayscreated","is_active"])

new_df1.to_csv("./pj_data/cleaned_data.csv", index=False)

In [145]:
new_df1.head()

,icoPriceUsd,currentStage,goalUsd,total_supply,price,exchangeName,n_contracts,isbnb,tags_reduce,iseth,issol,ismatic,isada,isavax,lifeday,dayscreated,is_active
0,0.08,1,98000.0,1.000000e+08,0.005751,10,1.0,1,6,0,0,0,0,0,656,656,1
1,0.03,1,80000.0,1.000000e+17,0.000000,10,2.0,0,10,1,1,0,0,0,311,311,0
2,0.60,1,1500000.0,1.000000e+08,0.055907,10,1.0,0,10,0,0,0,0,0,138,138,1
3,0.00,0,300000000.0,1.000000e+17,0.011018,3,1.0,0,10,0,0,0,0,0,4,4,1
4,0.95,1,500000.0,1.000000e+17,0.000000,3,1.0,0,10,1,0,0,0,0,70,70,0
